In [49]:
import nltk
import numpy as np
from sklearn.utils import shuffle

In [4]:
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression


In [11]:
nltk.download_shell()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> wordnet


        C:\Users\vinay.chavadi\AppData\Roaming\nltk_data...
      Unzipping corpora\wordnet.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


In [7]:
from bs4 import BeautifulSoup

In [8]:
wordnet_lemmetizer = WordNetLemmatizer()

In [14]:
stopwords=set(w.strip() for w in open('stopwords.txt'))

In [25]:
positive_reviews = BeautifulSoup(open('electronics/positive.review').read())

In [26]:
positive_reviews = positive_reviews.find_all('review_text')

In [29]:
negative_reviews = BeautifulSoup(open('electronics/negative.review').read())

In [34]:
t = positive_reviews[0]


In [17]:
len(positive_reviews)

1000

In [32]:
len(negative_reviews)

1000

In [77]:
negative_reviews[1]

<review_text>
It's a nice look, but it tips over very easily. It is not steady on a rug surface dispite what the picture on the box shows. My advice is if you need a CD rack that holds a lot of CD's? Save your money and invest in something nicer and more sturdy
</review_text>

In [37]:
np.random.shuffle(positive_reviews)

In [38]:
positive_reviews = positive_reviews[:len(negative_reviews)]

In [39]:
len(positive_reviews)

1000

In [40]:
def my_tokenizer(s):
    s=s.lower()
    #tokenize the sentence
    tokens = nltk.tokenize.word_tokenize(s)
    #consider the words whether the length > 2
    tokens =[t for t in tokens if len(t) > 2]
    tokens = [wordnet_lemmetizer.lemmatize(t) for t in tokens]
    tokens = [t for t in tokens if t not in stopwords]
    return tokens

In [41]:
word_index_map ={}
curren_index=0
positive_tokenized=[]
negative_tokenized=[]
for review in positive_reviews:
    tokens = my_tokenizer(review.text)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = curren_index
            curren_index+=1
    

In [42]:
for review in negative_reviews:
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = curren_index
            curren_index+=1

In [43]:
print("len(word_index_map):", len(word_index_map))

len(word_index_map): 11078


In [44]:
# now let's create our input matrices
def tokens_to_vector(tokens, label):
    x = np.zeros(len(word_index_map) + 1) # last element is for the label
    for t in tokens:
        i = word_index_map[t]
        x[i] += 1
    x = x / x.sum() # normalize it before setting label
    x[-1] = label
    return x
   

In [45]:
N = len(positive_tokenized) + len(negative_tokenized)
# (N x D+1 matrix - keeping them together for now so we can shuffle more easily later
data = np.zeros((N, len(word_index_map) + 1))
i = 0
for tokens in positive_tokenized:
    xy = tokens_to_vector(tokens, 1)
    data[i,:] = xy
    i += 1

In [46]:
for tokens in negative_tokenized:
    xy = tokens_to_vector(tokens, 0)
    data[i,:] = xy
    i += 1


In [51]:
# shuffle the data and create train/test splits
# try it multiple times!
np.random.shuffle(data)

X = data[:,:-1]
Y = data[:,-1]

In [53]:
Xtrain = X[:-100,]
Ytrain=Y[:-100,]
Xtest=X[-100:,]
Ytest=Y[-100:,]


In [54]:
model = LogisticRegression()

In [58]:
model.fit(Xtrain, Ytrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [59]:
print('The model Classification Rate is: ', model.score(Xtest,Ytest))

The model Classification Rate is:  0.64


In [61]:
from sklearn.naive_bayes import MultinomialNB

In [62]:
model = MultinomialNB()

In [63]:
model.fit(Xtrain, Ytrain)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [64]:
print('The model Classification Rate is: ', model.score(Xtest,Ytest))

The model Classification Rate is:  0.7


In [69]:
from sklearn.ensemble import RandomForestClassifier

In [70]:
model = RandomForestClassifier(n_estimators=200)

In [71]:
model.fit(Xtrain, Ytrain)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [72]:
print('The model Classification Rate is: ', model.score(Xtest,Ytest))

The model Classification Rate is:  0.71


In [73]:
from sklearn.ensemble import AdaBoostClassifier

In [74]:
model = AdaBoostClassifier(n_estimators=200)

In [75]:
model.fit(Xtrain, Ytrain)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=200, random_state=None)

In [76]:
print('The model Classification Rate is: ', model.score(Xtest,Ytest))

The model Classification Rate is:  0.71


In [19]:
str = "don't worry !! be happy?? 26th Jan. This is'nt new sentence. This is not the way."
tkns = my_tokenizer(str)

In [20]:
len(tkns)

9

In [21]:
tkns

["n't", 'worry', 'happy', '26th', 'jan.', 'this', "is'nt", 'sentence', 'this']

In [36]:
nltk.tokenize.word_tokenize(str)

['do',
 "n't",
 'worry',
 '!',
 '!',
 'be',
 'happy',
 '?',
 '?',
 '26th',
 'Jan',
 '.',
 'This',
 "is'nt",
 'new',
 'sentence',
 '.',
 'This',
 'is',
 'not',
 'the',
 'way',
 '.']